### install packages

In [1]:
!pip install powerlaw
import powerlaw
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.stats import kurtosis

In [2]:
# List of stock tickers to download
ticker_list = ['AAPL', 'GOOGL', 'MSFT']

# Dictionary to store dataframes
df_dict = {}

# Download and format stock data
for ticker in ticker_list:
    # Download data
    df = yf.download(ticker, start='2010-01-01', end='2023-12-01')

    # Remove rows with NaN values and fill any gaps by carrying forward the last known value
    df.dropna(inplace=True)
    df.fillna(method='ffill', inplace=True)

    # Calculate log returns and keep only that column
    df['Log Return'] = np.log(df['Close'] / df['Close'].shift(1))
    df = df[['Log Return']].dropna()  # Drop the first row as it will be NaN

    # Store the data with only log returns
    df_dict[ticker] = df

for name, df in df_dict.items():
    print(f"Table for: {name}")
    print(df.head().to_string())

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Table for: AAPL
            Log Return
Date                  
2010-01-05    0.001727
2010-01-06   -0.016034
2010-01-07   -0.001850
2010-01-08    0.006626
2010-01-11   -0.008861
Table for: GOOGL
            Log Return
Date                  
2010-01-05   -0.004413
2010-01-06   -0.025532
2010-01-07   -0.023555
2010-01-08    0.013243
2010-01-11   -0.001513
Table for: MSFT
            Log Return
Date                  
2010-01-05    0.000323
2010-01-06   -0.006156
2010-01-07   -0.010454
2010-01-08    0.006873
2010-01-11   -0.012802


###Kurtosis

In [3]:
for ticker in ticker_list:
    df = df_dict[ticker]

    # print results
    print(ticker)
    print("-"*len(ticker))
    print("kurtosis = " + str(kurtosis(df.iloc[:,0], fisher=True)))
    print("")


AAPL
----
kurtosis = 5.395563333704075

GOOGL
-----
kurtosis = 7.647265763502716

MSFT
----
kurtosis = 7.974291340094547



###Taleb's Kappa Metric
 https://arxiv.org/abs/1802.05495

In [4]:
def mean_abs_deviation(S):
    """
        Computation of mean absolute deviation of an input sample S
    """

    M = np.mean(np.abs(S - np.mean(S)))

    return M

In [5]:
def generate_n_sample(X,n):
    """
        Function to generate n random samples of size len(X) from an array X
    """

    # initialize sample
    S_n=0

    for i in range(n):

        # ramdomly sample len(X) observations from X and add it to the sample
        S_n = S_n + X[np.random.randint(len(X), size=int(np.round(len(X))))]

    return S_n

In [6]:
def kappa(X,n):
    """
        Taleb's kappa metric from n0=1 as described here: https://arxiv.org/abs/1802.05495

        Note: K_1n = kappa(1,n) = 2 - ((log(n)-log(1))/log(M_n/M_1)), where M_n denotes the mean absolute deviation of the sum of n random samples
    """

    S_1 = X
    S_n = generate_n_sample(X,n)

    M_1 = mean_abs_deviation(S_1)
    M_n = mean_abs_deviation(S_n)

    K_1n = 2 - (np.log(n)/np.log(M_n/M_1))

    return K_1n

In [7]:
n = 100 # number of samples to include in kappa calculation

for ticker in ticker_list:
    df = df_dict[ticker]

    # print results
    print(ticker)
    print("-"*len(ticker))
    print("kappa_1n = " + str(kappa(df.iloc[:,0].to_numpy(), n)))
    print("")

AAPL
----
kappa_1n = 0.1000010403824716

GOOGL
-----
kappa_1n = 0.12688032800940552

MSFT
----
kappa_1n = 0.10360562633595283



###compute mean k(1,100) of running calculation 5000x times

In [8]:
num_runs = 5000

kappa_dict = {}

for ticker in ticker_list:
    df = df_dict[ticker]

    kappa_list = []
    for i in range(num_runs):
        kappa_list.append(kappa(df.iloc[:,0].to_numpy(), n))


    kappa_dict[ticker] = np.array(kappa_list)

    print(ticker)
    print("-"*len(ticker))
    print("mean kappa_1n = " + str(np.mean(kappa_dict[ticker])))
    print("")

AAPL
----
mean kappa_1n = 0.09884989471043883

GOOGL
-----
mean kappa_1n = 0.12129985620482725

MSFT
----
mean kappa_1n = 0.11093688202298997

